# Importar librerías

In [2]:
# Pandas para manejo de DFs
import pandas as pd

# Manejo de vectores y operaciones matemáticas
import numpy as np

# JSON para leer el archivo de datos "Noticias.json"
import json

# NLTK
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

# LDA
from sklearn.decomposition import LatentDirichletAllocation
# CountVectorizer para obtener BoW y matriz rasgos-documentos con pesado binario o TF
from sklearn.feature_extraction.text import CountVectorizer
# TfidfVectorizer para obtener BoW y matriz rasgos-documentos con pesado TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Set de stopwords en español
spanish_sw = set(stopwords.words("spanish"))

---
# Ejercicio 1
- Dado un fichero en formato JSON con noticias se quiere obtener los diferentes topics utilizando LDA. En este primer ejercicio se proporciona el notebook completo hacerlo, los pasos son los siguientes:
    - Lectura del fichero y carga de datos.
    - Preprocesamiento básico de los textos.
    - Entrenamiento del algoritmo Latent Dirichlet Allocation de sklearn. Se recomienda ver la documentación de la librería para ver los diferentes parámetros del algoritmo.
    - Visualización de resultados.
    - Evaluación de los resultados.  

    Se pide ejecutar el código proporcionado y realizar ajustes sobre él para ver cómo
    cambian los resultados:
    - Cambiar el número de topics, ahora está a 2, pero revisando el fichero con las
    noticias, en realidad serían más.
    - Cambiar el preprocesamiento de los textos, eliminando stopwords al menos y lo
    que se considere para ver si mejoran los resultados.  
    
    ¿Cómo impacta en los resultados los diferentes cambios de los apartados anteriores?

---
## Lectura del fichero y carga de datos

In [3]:
# Carga del fichero "Noticias.json"
with open("data/Noticias.json", "r") as f:
    noticias = json.load(f)

df_noticias = pd.DataFrame(noticias)
print("DataFrame de noticias:")
df_noticias.head()

DataFrame de noticias:


,Title,TextContent
0,Suspendido el partido Villarreal-Espanyol por ...,El temporal de lluvia y nieve afecta a áreas d...
1,Reino Unido y otros países aliados de Ucrania ...,Los países europeos de la OTAN y Canadá han de...
2,Los premios Oscar dan la gloria al cine indie ...,¿Qué premia exactamente Hollywood y su industr...
3,"Emilia Pérez, Karla Sofía Gascón, Demi Moore y...","Fue Beckett el que, en un arrebato no precisam..."
4,La Aemet retira también el aviso rojo por fuer...,La Agencia Estatal de Meteorología (Aemet) ha ...


---
## Preprocesamiento básico de los datos
Inicialmente, únicamente tokenizaremos en sentencias y palabras y eliminaremos signos de puntuación mediante el uso de expresiones regulares (utilizando `regexp_tokenize()` de NLTK).

In [4]:
# Función para el preprocesamiento de los textos
def preprocess_text(text: str) -> str:
    # La ER devolverá palabras y números de 3 o más caracteres y/o dígitos
    word_tokenized = regexp_tokenize(text, r"[a-zA-Z0-9áéíóúÁÉÍÓÚ]{3,}")
    # Convertimos las palabras a minúsculas
    word_tokenized = " ".join([w.lower() for w in word_tokenized])
    return word_tokenized

# Aplicamos el preprocesamiento a los textos del DF
texts = df_noticias["TextContent"].copy().apply(preprocess_text)
texts.head()

0    temporal lluvia nieve afecta áreas diez provin...
1    los países europeos otan canadá han decidido a...
2    qué premia exactamente hollywood industria cua...
3    fue beckett que arrebato precisamente entusias...
4    agencia estatal meteorología aemet levantado s...
Name: TextContent, dtype: object

In [5]:
tf_vectorizer = CountVectorizer()
bow_encoded = tf_vectorizer.fit_transform(texts)
vocabulary = tf_vectorizer.vocabulary_
dictionary = tf_vectorizer.get_feature_names_out()

print("- Estadísticas de los datos (textos) tras el preprocesamiento:")
print(f"    - Tamaño del vocabulario: {len(vocabulary)} palabras únicas.")
print(f"    - Dimensiones de la matriz rasgos-documentos: {bow_encoded.shape}")

word_freq = bow_encoded.sum(axis=0).A1
top_words_idx = word_freq.argsort()[::-1][:10]
print("\n- 10 palabras más frecuentes:")
for word_idx in top_words_idx:
    print(f"    - {dictionary[word_idx]}: {word_freq[word_idx]}")

- Estadísticas de los datos (textos) tras el preprocesamiento:
    - Tamaño del vocabulario: 388 palabras únicas.
    - Dimensiones de la matriz rasgos-documentos: (12, 388)

- 10 palabras más frecuentes:
    - que: 15
    - los: 12
    - para: 9
    - del: 9
    - una: 8
    - por: 8
    - las: 8
    - con: 7
    - han: 6
    - más: 5


---
## Entrenamiento del algoritmo LDA (*Latent Dirichlet Allocation*)

In [6]:
# Establecer parámetros del algoritmo
n_topics = 2
alpha = 1.0
beta = 0.1

print("Configuración del modelo LDA:")
print(f"- Número de tópicos: {n_topics}")
print(f"- Alpha: {alpha}")
print(f"- Beta: {beta}")
print("\nIniciando entrenamiento...\n")

lda = LatentDirichletAllocation(
    n_components=n_topics,
    doc_topic_prior=alpha,
    topic_word_prior=beta,
    max_iter=25,
    learning_method="online",
    evaluate_every=1,
    n_jobs=-1,
    random_state=0,
    verbose=True
)

lda.fit(bow_encoded)

Configuración del modelo LDA:
- Número de tópicos: 2
- Alpha: 1.0
- Beta: 0.1

Iniciando entrenamiento...

iteration: 1 of max_iter: 25, perplexity: 2748.3978
iteration: 2 of max_iter: 25, perplexity: 2426.2626
iteration: 3 of max_iter: 25, perplexity: 2116.8699
iteration: 4 of max_iter: 25, perplexity: 1878.4621
iteration: 5 of max_iter: 25, perplexity: 1699.9648
iteration: 6 of max_iter: 25, perplexity: 1561.9508
iteration: 7 of max_iter: 25, perplexity: 1452.3153
iteration: 8 of max_iter: 25, perplexity: 1363.4705
iteration: 9 of max_iter: 25, perplexity: 1290.4080
iteration: 10 of max_iter: 25, perplexity: 1229.6602
iteration: 11 of max_iter: 25, perplexity: 1178.7285
iteration: 12 of max_iter: 25, perplexity: 1135.7519
iteration: 13 of max_iter: 25, perplexity: 1099.3065
iteration: 14 of max_iter: 25, perplexity: 1068.2785
iteration: 15 of max_iter: 25, perplexity: 1041.7801
iteration: 16 of max_iter: 25, perplexity: 1019.0935
iteration: 17 of max_iter: 25, perplexity: 999.6307
it

LatentDirichletAllocation(doc_topic_prior=1.0, evaluate_every=1,
                          learning_method='online', max_iter=25, n_components=2,
                          n_jobs=-1, random_state=0, topic_word_prior=0.1,
                          verbose=True)

---
## Análisis de resultados

---
### Palabras más relevantes por tópico

In [7]:
# Establecer parámetros de visualización
no_top_words = 10
no_top_documents = 2

# Obtener distribuciones
documents_topics_matrix = lda.transform(bow_encoded)
topics_words_matrix = lda.components_

In [8]:
print("- Palabras más relevantes por tópico:")
for topic_idx, topic in enumerate(topics_words_matrix):
    print(f"    - Tópico {topic_idx+1}:")
    top_words_idx = topic.argsort()[::-1][:no_top_words]
    top_words = [dictionary[i] for i in top_words_idx]
    top_probs = [topic[i] for i in top_words_idx]
    for word, prob in zip(top_words, top_probs):
        print(f"        - {word}: {prob:.4f}")
    print()

- Palabras más relevantes por tópico:
    - Tópico 1:
        - que: 11.5840
        - del: 7.8160
        - para: 6.8395
        - los: 6.7493
        - las: 5.9023
        - por: 4.8604
        - este: 3.9948
        - han: 3.9760
        - más: 3.9547
        - países: 3.9524

    - Tópico 2:
        - los: 5.0313
        - una: 4.0576
        - por: 3.0860
        - con: 3.0731
        - que: 3.0707
        - verdad: 2.0998
        - para: 2.0605
        - han: 2.0536
        - las: 2.0469
        - cuando: 2.0444



---
### Documentos más relevantes por tópico

In [9]:
print("- Documentos más representativos por tópico (con más peso en cada tópico):")
for topic_idx in range(n_topics):
    print(f"    - Tópico {topic_idx+1}:")
    top_documents_idx = documents_topics_matrix[:, topic_idx].argsort()[::-1][:no_top_documents]

    for document_idx in top_documents_idx:
        title = df_noticias.iloc[document_idx]["Title"]
        weight = documents_topics_matrix[document_idx, topic_idx]
        print(f"        - Título de la noticia: {title}")
        print(f"            - Peso en el tópico: {weight:.4f}")
    print()

- Documentos más representativos por tópico (con más peso en cada tópico):
    - Tópico 1:
        - Título de la noticia: Suspendido el partido Villarreal-Espanyol por la emergencia meteorológica.
            - Peso en el tópico: 0.9844
        - Título de la noticia: Reino Unido y otros países aliados de Ucrania se comprometen a rearmar a Zelenski: 'Botas en el terreno y aviones en los cielos'
            - Peso en el tópico: 0.9840

    - Tópico 2:
        - Título de la noticia: Los premios Oscar dan la gloria al cine indie y castigan una vez más a Netflix
            - Peso en el tópico: 0.9865
        - Título de la noticia: Los científicos abandonan el sueño de crear ‘vida espejo’, que podría convertirse en pesadilla
            - Peso en el tópico: 0.9524



---
### Distribución de tópicos en documentos específicos

In [10]:
print("- Distribución de tópicos en cada documento:")
titles = df_noticias["Title"].copy()
topic_names = [f"Topic {topic_idx}" for topic_idx in range(n_topics)]
topics_documents_distribution_df = pd.DataFrame(documents_topics_matrix.copy(), columns=topic_names, index=titles.tolist())
topics_documents_distribution_df.head()

- Distribución de tópicos en cada documento:


,Topic 0,Topic 1
Suspendido el partido Villarreal-Espanyol por la emergencia meteorológica.,0.984434,0.015566
Reino Unido y otros países aliados de Ucrania se comprometen a rearmar a Zelenski: 'Botas en el terreno y aviones en los cielos',0.984037,0.015963
Los premios Oscar dan la gloria al cine indie y castigan una vez más a Netflix,0.013491,0.986509
"Emilia Pérez, Karla Sofía Gascón, Demi Moore y la decencia, a la cabeza de los perdedores de la noche",0.975620,0.024380
"La Aemet retira también el aviso rojo por fuertes lluvias en Castellón, que se mantiene en naranja.",0.976819,0.023181


---
### Distribución de palabras en cada tópico

In [11]:
print("- Distribución de palabras por tópico:")
words = dictionary.copy()
topic_names = [f"Topic {topic_idx}" for topic_idx in range(n_topics)]
topics_words_matrix_normalized = topics_words_matrix.copy() / topics_words_matrix.sum(axis=1).reshape(-1, 1)
topics_words_distribution_df = pd.DataFrame(topics_words_matrix_normalized, columns=words, index=topic_names)
topics_words_distribution_df.head()

- Distribución de palabras por tópico:


,000,100,106,143,180,2017,2024,500,abundancia,academia,...,willem,zelenski,zonas,zorrilla,ánimos,área,áreas,éxito,única,único
Topic 0,0.002564,0.00255,0.002564,0.002527,0.002543,0.000345,0.000350,0.002535,0.002550,0.000345,...,0.000332,0.002563,0.002536,0.000342,0.002548,0.002508,0.002534,0.002580,0.000323,0.002521
Topic 1,0.000720,0.00071,0.000733,0.000773,0.000716,0.005502,0.005496,0.000753,0.000713,0.005492,...,0.005470,0.000705,0.000741,0.005535,0.000694,0.000774,0.000742,0.000714,0.005504,0.000751


---
## Evaluación del modelo

In [12]:
log_likelihood = lda.score(bow_encoded)
perplexity = lda.perplexity(bow_encoded)

print("- Métricas de evaluación:")
print(f"    - Log-likelihood (cuanto más alto, mejor): {log_likelihood}")
print(f"    - Perplexity (cuanto más bajo, mejor): {perplexity}")

- Métricas de evaluación:
    - Log-likelihood (cuanto más alto, mejor): -3696.0522494033266
    - Perplexity (cuanto más bajo, mejor): 915.3299423364903


---
### Probar diferentes `n_topics` para LDA

In [13]:
n_topics_range = [4, 6, 8]
results = []

for n_t in n_topics_range:
    print(f"\n\nComenzando entrenamiento con {n_t} tópicos...")
    lda = LatentDirichletAllocation(
        n_components=n_t,
        doc_topic_prior=alpha,
        topic_word_prior=beta,
        max_iter=25,
        random_state=0,
        learning_method="online",
        verbose=True
    )
    lda.fit(bow_encoded)

    results.append({
        "n_topics": n_t,
        "log-likelihood": lda.score(bow_encoded),
        "perplexity": lda.perplexity(bow_encoded)
    })

results_df = pd.DataFrame(results)
results_df



Comenzando entrenamiento con 4 tópicos...
iteration: 1 of max_iter: 25
iteration: 2 of max_iter: 25
iteration: 3 of max_iter: 25
iteration: 4 of max_iter: 25
iteration: 5 of max_iter: 25
iteration: 6 of max_iter: 25
iteration: 7 of max_iter: 25
iteration: 8 of max_iter: 25
iteration: 9 of max_iter: 25
iteration: 10 of max_iter: 25
iteration: 11 of max_iter: 25
iteration: 12 of max_iter: 25
iteration: 13 of max_iter: 25
iteration: 14 of max_iter: 25
iteration: 15 of max_iter: 25
iteration: 16 of max_iter: 25
iteration: 17 of max_iter: 25
iteration: 18 of max_iter: 25
iteration: 19 of max_iter: 25
iteration: 20 of max_iter: 25
iteration: 21 of max_iter: 25
iteration: 22 of max_iter: 25
iteration: 23 of max_iter: 25
iteration: 24 of max_iter: 25
iteration: 25 of max_iter: 25


Comenzando entrenamiento con 6 tópicos...
iteration: 1 of max_iter: 25
iteration: 2 of max_iter: 25
iteration: 3 of max_iter: 25
iteration: 4 of max_iter: 25
iteration: 5 of max_iter: 25
iteration: 6 of max_iter: 

,n_topics,log-likelihood,perplexity
0,4,-3686.493134,899.328027
1,6,-3762.340227,1034.410662
2,8,-3765.308098,1040.090401


---
## Añadimos preprocesamiento a los textos

In [14]:
# Función auxiliar para obtener los tags de WN a partir de los de NLTK
def get_wn_tag(tag: str) -> str:
    nltk2wn = {
        "N": wn.NOUN,
        "V": wn.VERB,
        "J": wn.ADJ,
        "R": wn.ADV
    }
    if tag[0] in nltk2wn.keys():
        return nltk2wn[tag[0]]
    else:
        return None 

# Función para preprocesar cada texto
def preprocess_text(text: str) -> str:
    word_tokenized = regexp_tokenize(text, r"[a-zA-Z0-9áéíóúÁÉÍÓÚ]{3,}")
    pos_tags = nltk.pos_tag(word_tokenized)
    lemmatized = [WordNetLemmatizer().lemmatize(w.lower(), get_wn_tag(tag)) for w, tag in pos_tags if w.lower() not in spanish_sw and get_wn_tag(tag) is not None]
    return " ".join(lemmatized)

# Aplicamos el preprocesamiento a los textos
preprocessed_texts = df_noticias["TextContent"].copy().apply(preprocess_text)
print("- Textos preprocesados (eliminación de puntuación + eliminación de SWs + lematización):")
preprocessed_texts.head()

- Textos preprocesados (eliminación de puntuación + eliminación de SWs + lematización):


0    temporal lluvia nieve afecta áreas diez provin...
1    países europeos otan canadá decidido aumentar ...
2    premia exactamente hollywood industria disting...
3    beckett arrebato precisamente dijo aquello int...
4    agencia estatal meteorología aemet levantado n...
Name: TextContent, dtype: object

In [15]:
vectorizer = CountVectorizer()
bow_encoded = vectorizer.fit_transform(preprocessed_texts)
vocabulary = vectorizer.vocabulary_
dictionary = vectorizer.get_feature_names_out()

---
### Volvemos a entrenar el modelo con diferentes configuraciones de los hiperparámetros

In [ ]:
n_topics_range = [4, 6, 8]
results = []

for n_t in n_topics_range:
    print(f"\n\nComenzando entrenamiento con {n_t} tópicos...")
    lda = LatentDirichletAllocation(
        n_components=n_t,
        doc_topic_prior=0.2,
        topic_word_prior=0.5,
        max_iter=25,
        random_state=0,
        learning_method="online",
        verbose=True
    )
    lda.fit(bow_encoded)

    topics_words_matrix = lda.components_
    documents_topics_matrix = lda.transform(bow_encoded)
    top_words = []
    top_documents = []
    for topic_idx in range(n_t):
        # Obtener 10 palabras más probables en cada tópico
        top_words_idx = topics_words_matrix[topic_idx, :].argsort()[::-1][:10]
        top_words.append((f"Topic {topic_idx+1}", [dictionary[i] for i in top_words_idx]))

        # Obtener 2 documentos más probables en cada tópico
        top_documents_idx = documents_topics_matrix[:, topic_idx].argsort()[::-1][:2]
        top_documents.append((f"Topic {topic_idx+1}", [df_noticias.iloc[document_idx]["Title"] for document_idx in top_documents_idx]))

    results.append({
        "n_topics": n_t,
        "log-likelihood": lda.score(bow_encoded),
        "perplexity": lda.perplexity(bow_encoded),
        "topwords": top_words,
        "topdocs": top_documents
    })

    

results_df = pd.DataFrame(results)
results_df



Comenzando entrenamiento con 4 tópicos...
iteration: 1 of max_iter: 25
iteration: 2 of max_iter: 25
iteration: 3 of max_iter: 25
iteration: 4 of max_iter: 25
iteration: 5 of max_iter: 25
iteration: 6 of max_iter: 25
iteration: 7 of max_iter: 25
iteration: 8 of max_iter: 25
iteration: 9 of max_iter: 25
iteration: 10 of max_iter: 25
iteration: 11 of max_iter: 25
iteration: 12 of max_iter: 25
iteration: 13 of max_iter: 25
iteration: 14 of max_iter: 25
iteration: 15 of max_iter: 25
iteration: 16 of max_iter: 25
iteration: 17 of max_iter: 25
iteration: 18 of max_iter: 25
iteration: 19 of max_iter: 25
iteration: 20 of max_iter: 25
iteration: 21 of max_iter: 25
iteration: 22 of max_iter: 25
iteration: 23 of max_iter: 25
iteration: 24 of max_iter: 25
iteration: 25 of max_iter: 25


Comenzando entrenamiento con 6 tópicos...
iteration: 1 of max_iter: 25
iteration: 2 of max_iter: 25
iteration: 3 of max_iter: 25
iteration: 4 of max_iter: 25
iteration: 5 of max_iter: 25
iteration: 6 of max_iter: 

,n_topics,log-likelihood,perplexity,topwords,topdocs
0,4,-2376.519590,436.180093,"[(Topic 1, [litros, rojo, levantado, casa, cer...","[(Topic 1, [La Aemet retira también el aviso r..."
1,6,-2396.070932,458.545099,"[(Topic 1, [litros, rojo, levantado, cierren, ...","[(Topic 1, [La Aemet retira también el aviso r..."
2,8,-2384.519967,445.196849,"[(Topic 1, [litros, rojo, levantado, riesgo, d...","[(Topic 1, [La Aemet retira también el aviso r..."
